<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Why-ObsPy?" data-toc-modified-id="Why-ObsPy?-1">Why ObsPy?</a></span></li><li><span><a href="#How-is-the-seismogram-represented?" data-toc-modified-id="How-is-the-seismogram-represented?-2">How is the seismogram represented?</a></span></li><li><span><a href="#Receiver-metadata" data-toc-modified-id="Receiver-metadata-3">Receiver metadata</a></span></li><li><span><a href="#Data-structures-for-earthquakes" data-toc-modified-id="Data-structures-for-earthquakes-4">Data structures for earthquakes</a></span></li><li><span><a href="#Signal-processing-routines-obspy.signal" data-toc-modified-id="Signal-processing-routines-obspy.signal-5">Signal processing routines <code>obspy.signal</code></a></span></li><li><span><a href="#Resources" data-toc-modified-id="Resources-6">Resources</a></span></li></ul></div>

<h1 style="text-align: center"><span class="tocSkip"></span>Integrate Seismology Into the Scientific Python Ecosystem with ObsPy</h1>
<h2 style="text-align: center"><span class="tocSkip"></span>Shane Zhang*</h2>
<h3 style="text-align: center"><span class="tocSkip"></span>University of Colorado Boulder</h3>
<h3 style="text-align: center"><span class="tocSkip"></span>Sep 27, 2021</h3>
<h4 style="text-align: center"><span class="tocSkip"></span><a href="mailto:shane.zhang@colorado.edu">*shane.zhang@colorado.edu</a></h4>

# Why ObsPy?

- Python
- Open source with community-wide contributions
- Gaining popularity
- Common signal processing routines
- Interface for web services from data centers
- ...

Nothing in obspy makes sense except in the light of a **sensible representation of seismic data which provides "a bridge for seismology into the scientific Python ecosystem"** [Krischer et al., 2015](https://doi.org/10.1088/1749-4699/8/1/014003). These representations are (duly) called `core` classes in obspy (`obspy.core`).


<figure>
    <img src="fig/gh.png" class="center" style="width:100%">
    <figcaption><b>Fig: Development of ObsPy</b> (https://user-images.githubusercontent.com/1842780/70671351-0c884100-1c7c-11ea-81ed-7c477b7cf29c.png).</figcaption>
</figure>

Thus, in this tutorial, let's try to understand how seismic data (waveform, station response, catalogue, etc.) are respresented by obspy, and how it makes the transition to the scientific Python ecosystem seamless.

The organization of the sections are sorted by the *irreplaceability* of obspy functionalities (in my view). For instance, I believe the representation of waveforms is more essential than the representation of earthquake catalogues, and the I/O between various data formats and python objects is harder to be replaced by other softwares than, say, signal processing utilities.

Therefore, this tutorial is more of a high-level overview than a hands-on exercise about the specific applications (e.g., how to do this processing to the waveform, how to make a plot like this). The specific questions can be overwhelming in amount and will be overly biased by my personal experiences. They can be learned from the various [resources](#Resources).

- Where does ObsPy stand in the scientific Python ecosystem?

<figure>
    <img src="fig/Harris20_2.png" class="center" style="width:80%">
    <figcaption><b>Fig: Hierarchy of scientific python ecosystem. </b> [Harris et al., 2020](https://doi.org/10.1038/s41586-020-2649-2).</figcaption>
</figure>

# How is the seismogram represented?

<figure>
    <img src="fig/Krischer15_1.png" class="center" style="width:100%">
    <figcaption><b>Fig: Structure of a `Trace`, the most fundatmental object.</b> [Krischer et al., 2015](https://doi.org/10.1088/1749-4699/8/1/014003)</figcaption>
</figure>


- The fundamental observable in seismology is the seismogram, a record of mechanical oscillations of the planet.
- Everything in Python is an `object`, something can have `properties` (attributes) and `methods` (functions).

In [ ]:
import matplotlib.pyplot as plt
import obspy


plt.rcParams.update({
    'figure.dpi': 200,
})

In [ ]:
st = obspy.read()
tr = st[0] 

- Metadata is saved in `stats`, like a python `dict`.

In [ ]:
tr.stats

In [ ]:
# Both will work
tr.stats['station']
# tr.stats.delta

- The time series itself is a `numpy` array.

In [ ]:
tr.data

In [ ]:
type(tr.data)
# tr.data.mean()

In [ ]:
dir(tr)

In [ ]:
tr.plot();

In [ ]:
fig, ax = plt.subplots()
ax.plot(tr.times(), tr.data)

- Supported formats
    - `read`: **MSEED**, **SAC**, SEG2, SEGY, SEISAN, SU, WIN, MATLAB (via Scipy) ...
    - `write`: a subset of `read`

The seismic wavefield is a vector field in the 4-D spacetime, and thus we have to move beyond a `Trace`, a 1-D time series in essence, which only records oscillations along a certain direction at a location fixed to Earth (Lagrangian reference frame). For instance, it can make sense to group different components of the seismograms at the same location together, or to remove responses across certain types of instruments.

A "group" of `Trace`(s) is a `Stream`, which is somewhat similar to a `list` in python.

In [ ]:
st

In [ ]:
st.plot();

In [ ]:
dir(st)

# Receiver metadata

<figure>
    <img src="fig/Inventory.png" class="center" style="width:100%">
    <figcaption><b>Fig: Data structure of receiver information.</b> (https://docs.obspy.org/packages/obspy.core.html)</figcaption>
</figure>

An `inventory` is structured after [StationXML](https://www.fdsn.org/xml/station/). Obspy supports I/O for
- `read_inventory`: RESP, SEED, STATIONTXT, STATIONXML, ...
- `write`: KML, SACPZ, STATIONTXT, STATIONXML, ...

In [ ]:
inv = obspy.read_inventory()

In [ ]:
inv

In [ ]:
inv.networks[1].stations[0].channels[0].response.plot(min_freq=.001);

Comment: Such a structure makes sense for the currently most common type of instruments which are 3-component (3-C) seismometers spatially isolated from each other. New types of receivers, such as 6-C seismometers and distributed acoustic sensors (DAS) may call for new designs.

# Data structures for earthquakes

<figure>
    <img src="fig/Event.png" class="center" style="width:100%">
    <figcaption><b>Fig: Data structure for earthquakes.</b> (https://docs.obspy.org/packages/obspy.core.html)</figcaption>
</figure>

A `catalog` is structured after [QuakeML](https://quake.ethz.ch/quakeml/). Obspy supports I/O for
- `read_events`: CMTSOLUTION, FOCMEC, HYPODDPHA, NDK, QUAKEML, ...
- `write`: KML, SHAPEFILE, ...

In [ ]:
catalog = obspy.read_events()
catalog

In [ ]:
catalog[0]

In [ ]:
catalog[0].origins

In [ ]:
catalog[0].magnitudes

Comment: With the emergence of ambient noise seismology, specific applications may not use tectonic earthquakes as the dominante source of energy.

# Signal processing routines `obspy.signal`

In [ ]:
# Note the operation is in place
tr_filtered = tr.copy().filter(type='bandpass', freqmin=1, freqmax=2)
tr_filtered.plot();

In [ ]:
# Provenance is logged for *reproducibility*
tr_filtered.stats.processing

# Resources

- Other niceties that cannot be covered in this tutorial
    - `obspy.clients`: interface for data center web services (request, downloading, computation, etc.)
    - `obspy.geodetics`
    - `obspy.imaging`: beachball, map
    - `obspy.taup`
    - ...

- Learning resources
    - [Official wiki](https://github.com/obspy/obspy/wiki)
    - [YouTube videos](https://www.youtube.com/results?search_query="obspy")
    - IPython notebooks in [seismo-live](http://seismo-live.org/)
    